In [355]:
import requests
import json
import pandas as pd
import time
from datetime import datetime, timezone
import re
from math import cos,radians
import psycopg2


import warnings
warnings.filterwarnings('ignore')

In [1]:
key = 'a1a0721d8c62d9ec'

### For hourly forecast

In [139]:
hourly = requests.get('http://api.wunderground.com/api/a1a0721d8c62d9ec/hourly/q/WA/Issaquah.json')
hourly.status_code




200

In [140]:
prediction = json.loads(hourly.content)

In [130]:
prediction.keys()

dict_keys(['response', 'hourly_forecast'])

### For current conditions

In [33]:
current = requests.get('http://api.wunderground.com/api/a1a0721d8c62d9ec/conditions/q/WA/Issaquah.json')
current.status_code

200

In [36]:
current = json.loads(current.content)

### Getting the feed in the same format of the model

In [41]:
features = ['overall_rank_0','overall_rank_1', 'overall_rank_2', 
            'alti_0', 'alti_4', 'alti_8', 'alti_12', 'alti_16',
            'drct_0', 'drct_4', 'drct_8', 'drct_12', 'drct_16',
            'dwpf_0', 'dwpf_4', 'dwpf_8', 'dwpf_12', 'dwpf_16',
            'p01i_0', 'p01i_4', 'p01i_8', 'p01i_12', 'p01i_16',
            'relh_0', 'relh_4', 'relh_8', 'relh_12', 'relh_16',
            'sknt_0', 'sknt_4', 'sknt_8', 'sknt_12', 'sknt_16',
            'skyc1_0', 'skyc1_4', 'skyc1_8', 'skyc1_12', 'skyc1_16',
            'psgr_0', 'psgr_4', 'psgr_8', 'psgr_12', 'psgr_16',
            'tmpf_0', 'tmpf_4', 'tmpf_8', 'tmpf_12', 'tmpf_16']

In [263]:
TEST = pd.DataFrame(prediction['hourly_forecast'])


In [301]:
output = []
for i,item in enumerate(prediction['hourly_forecast']):
    
    output.append({'epoch':item['FCTTIME']['epoch'],
                   'hour':item['FCTTIME']['hour'],
                    'alti':item['mslp']['english'],
                    'drct': item['wdir']['degrees'],
                    'dwpf': item['dewpoint']['english'],
                    'p01i' : item ['qpf']['english'],
                    'relh':item ['humidity'],
                    'sknt':int(item['wspd']['metric'])/1.8 ,
                    'skyc1':item['sky'],
                    'tmpf':item['temp']['english']})


In [303]:
TEMP = pd.DataFrame(output, columns = ['epoch','hour','alti','drct','dwpf','p01i','relh','sknt','skyc1','psgr','tmpf'])
TEMP.head()

,epoch,hour,alti,drct,dwpf,p01i,relh,sknt,skyc1,psgr,tmpf
0,1517331600,9,30.08,198,40,0.0,90,10.0,70,NaN,43
1,1517335200,10,30.09,204,40,0.0,86,10.0,68,NaN,44
2,1517338800,11,30.1,208,40,0.01,82,10.0,66,NaN,45
3,1517342400,12,30.1,206,40,0.02,81,10.0,73,NaN,46
4,1517346000,13,30.1,205,40,0.0,80,10.0,72,NaN,46


In [304]:
# create timespamp
def ts(epoch):
    return datetime.fromtimestamp(int(epoch)).strftime('%Y-%m-%d %H:%M:%S')

In [305]:
TEMP['TS'] = TEMP['epoch'].apply(ts)

In [306]:
TEMP['TS'] = pd.to_datetime(TEMP['TS'])

In [307]:
# Assigning as index
TEMP.set_index('TS',inplace=True)

In [309]:
# converting strings to numbers

lst = ['alti','drct','dwpf','p01i','relh','sknt','skyc1','tmpf']
for col in lst:
    TEMP[col] = TEMP[col].apply(pd.to_numeric, errors='coerce')

In [310]:
# Aggregating in 4H periods

In [312]:
lst = ['alti','drct','dwpf','p01i','relh','sknt','skyc1']
BASE = pd.DataFrame(TEMP.tmpf.resample('4H').mean())

for col in lst:
    ADD = pd.DataFrame(TEMP[col].resample('4H').mean())
    frames = [BASE,ADD]
    RDF = pd.concat(frames,axis=1)
    BASE = RDF

RDF.head()

,tmpf,alti,drct,dwpf,p01i,relh,sknt,skyc1
TS,,,,,,,,
2018-01-30 08:00:00,44.00,30.0900,203.333333,40.00,0.003333,86.00,10.000000,68.00
2018-01-30 12:00:00,46.00,30.1050,204.750000,39.75,0.005000,79.00,10.000000,71.25
2018-01-30 16:00:00,44.25,30.1300,198.500000,38.50,0.000000,81.00,7.777778,64.25
2018-01-30 20:00:00,41.75,30.1675,192.500000,37.50,0.000000,84.75,6.111111,71.75
2018-01-31 00:00:00,40.75,30.1925,189.000000,37.50,0.000000,88.00,5.555556,89.50


In [273]:
# Creating pressure gradient

In [313]:
RDF['psgr'] = pd.DataFrame(RDF['alti'].resample('4H',label='press_gr').diff())

/Users/eduardodeangelis/anaconda2/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 
.resample() is now a deferred operation
You called diff(...) on this deferred object which materialized it into a series
by implicitly taking the mean.  Use .resample(...).mean() instead
  """Entry point for launching an IPython kernel.


In [314]:
# resetting the index to take the timestamp out
RDF.reset_index(inplace=True)

In [315]:
def get_date(ts):
    return ts.date()

def get_hour(ts):
    return ts.hour

RDF['date']=RDF['TS'].apply(get_date)
RDF['hour']=RDF['TS'].apply(get_hour)

In [316]:
RDF.head()

,TS,tmpf,alti,drct,dwpf,p01i,relh,sknt,skyc1,psgr,date,hour
0,2018-01-30 08:00:00,44.00,30.0900,203.333333,40.00,0.003333,86.00,10.000000,68.00,NaN,2018-01-30,8
1,2018-01-30 12:00:00,46.00,30.1050,204.750000,39.75,0.005000,79.00,10.000000,71.25,0.0150,2018-01-30,12
2,2018-01-30 16:00:00,44.25,30.1300,198.500000,38.50,0.000000,81.00,7.777778,64.25,0.0250,2018-01-30,16
3,2018-01-30 20:00:00,41.75,30.1675,192.500000,37.50,0.000000,84.75,6.111111,71.75,0.0375,2018-01-30,20
4,2018-01-31 00:00:00,40.75,30.1925,189.000000,37.50,0.000000,88.00,5.555556,89.50,0.0250,2018-01-31,0


In [318]:
#Getting one row per day - different rows become columns

RDF = RDF.pivot_table(index='date', 
                    columns='hour',
                    values=['tmpf','dwpf','relh','drct','sknt','p01i','alti','skyc1','psgr'])

In [319]:
#Flattening the multindex
RDF = pd.DataFrame(RDF.to_records())
RDF.head()

,date,"('alti', 0)","('alti', 4)","('alti', 8)","('alti', 12)","('alti', 16)","('alti', 20)","('drct', 0)","('drct', 4)","('drct', 8)",...,"('skyc1', 8)","('skyc1', 12)","('skyc1', 16)","('skyc1', 20)","('tmpf', 0)","('tmpf', 4)","('tmpf', 8)","('tmpf', 12)","('tmpf', 16)","('tmpf', 20)"
0,2018-01-30,NaN,NaN,30.0900,30.105,30.1300,30.1675,NaN,NaN,203.333333,...,68.0,71.25,64.25,71.75,NaN,NaN,44.00,46.0,44.25,41.75
1,2018-01-31,30.1925,30.21,30.2225,30.210,30.1975,30.2000,189.0,175.75,168.000000,...,96.0,97.75,98.75,99.00,40.75,40.25,41.75,44.0,43.50,43.00


In [320]:
mi = RDF.columns
mi = list(mi)
mi = mi[1:] #removing 'date' to make it easier

In [321]:
words = re.findall('(\w{4,10})', str(mi))
# hours = re.findall('\d{1,2}',str(mi))
hours = ['0','4','8','12','16','20'] * len(words)
idx = [w + '_' + h for w,h in zip(words,hours)]
idx = ['date']+idx

In [322]:
RDF.columns = idx

In [323]:
for col in idx:
    RDF[col]= RDF[col].interpolate(method='linear', axis=0).ffill().bfill()

In [324]:
RDF.head()

,date,alti_0,alti_4,alti_8,alti_12,alti_16,alti_20,drct_0,drct_4,drct_8,...,skyc1_8,skyc1_12,skyc1_16,skyc1_20,tmpf_0,tmpf_4,tmpf_8,tmpf_12,tmpf_16,tmpf_20
0,2018-01-30,30.1925,30.21,30.0900,30.105,30.1300,30.1675,189.0,175.75,203.333333,...,68.0,71.25,64.25,71.75,40.75,40.25,44.00,46.0,44.25,41.75
1,2018-01-31,30.1925,30.21,30.2225,30.210,30.1975,30.2000,189.0,175.75,168.000000,...,96.0,97.75,98.75,99.00,40.75,40.25,41.75,44.0,43.50,43.00


In [293]:
# transforming wind direction

def get_cos(direction):
    return (cos(radians(direction)))**2

In [325]:
direction = ['drct_0', 'drct_4', 'drct_8', 'drct_12', 'drct_16', 'drct_20']

for col in direction:
    RDF[col]=RDF[col].apply(get_cos)

In [326]:
RDF.head()

,date,alti_0,alti_4,alti_8,alti_12,alti_16,alti_20,drct_0,drct_4,drct_8,...,skyc1_8,skyc1_12,skyc1_16,skyc1_20,tmpf_0,tmpf_4,tmpf_8,tmpf_12,tmpf_16,tmpf_20
0,2018-01-30,30.1925,30.21,30.0900,30.105,30.1300,30.1675,0.975528,0.994508,0.843121,...,68.0,71.25,64.25,71.75,40.75,40.25,44.00,46.0,44.25,41.75
1,2018-01-31,30.1925,30.21,30.2225,30.210,30.1975,30.2000,0.975528,0.994508,0.956773,...,96.0,97.75,98.75,99.00,40.75,40.25,41.75,44.0,43.50,43.00


In [346]:
RDF = pd.concat([RDF]*3, ignore_index=True)

In [356]:
RDF['overall_rank_0']=0
RDF['overall_rank_1']=0
RDF['overall_rank_2']=0
RDF['overall_rank_0'][0]=1
RDF['overall_rank_0'][3]=1
RDF['overall_rank_1'][1]=1
RDF['overall_rank_1'][4]=1
RDF['overall_rank_2'][2]=1
RDF['overall_rank_2'][5]=1

### Function to get it all done in one step

In [387]:
def get_ready(api_forecast):
    
    '''Input: jspn response from API (.content)
       Ouput: DF ready to be used for predictions'''
    
    prediction = json.loads(api_forecast)
    output = []
    for item in prediction['hourly_forecast']:
        output.append({'epoch':item['FCTTIME']['epoch'],
                   'hour':item['FCTTIME']['hour'],
                    'alti':item['mslp']['english'],
                    'drct': item['wdir']['degrees'],
                    'dwpf': item['dewpoint']['english'],
                    'p01i' : item ['qpf']['english'],
                    'relh':item ['humidity'],
                    'sknt':int(item['wspd']['metric'])/1.8 ,
                    'skyc1':item['sky'],
                    'tmpf':item['temp']['english']})
    TEMP = pd.DataFrame(output, columns = ['epoch','hour','alti','drct','dwpf','p01i','relh','sknt','skyc1',
                                           'psgr','tmpf'])
    TEMP['TS'] = TEMP['epoch'].apply(ts)
    TEMP['TS'] = pd.to_datetime(TEMP['TS'])
    
    # Assigning as index
    TEMP.set_index('TS',inplace=True)
    
    # converting strings to numbers
    lst = ['alti','drct','dwpf','p01i','relh','sknt','skyc1','tmpf']
    for col in lst:
        TEMP[col] = TEMP[col].apply(pd.to_numeric, errors='coerce')
    
    # Aggregating in 4h periods
    lst = ['alti','drct','dwpf','p01i','relh','sknt','skyc1']
    BASE = pd.DataFrame(TEMP.tmpf.resample('4H').mean())
    for col in lst:
        ADD = pd.DataFrame(TEMP[col].resample('4H').mean())
        frames = [BASE,ADD]
        RDF = pd.concat(frames,axis=1)
        BASE = RDF
    
    # Creating pressure gradient
    RDF['psgr'] = pd.DataFrame(RDF['alti'].resample('4H',label='press_gr').diff())
    
    # resetting the index to take the timestamp out
    RDF.reset_index(inplace=True)
    
    #creating date and hore columns
    RDF['date']=RDF['TS'].apply(get_date)
    RDF['hour']=RDF['TS'].apply(get_hour)
    
    #Getting one row per day - different rows become columns
    RDF = RDF.pivot_table(index='date', 
                    columns='hour',
                    values=['tmpf','dwpf','relh','drct','sknt','p01i','alti','skyc1','psgr'])
    #Flattening the multindex
    RDF = pd.DataFrame(RDF.to_records())
    
    #removing 'date' to make it easier
    mi = RDF.columns
    mi = list(mi)
    mi = mi[1:]
    
    # putting names in the same format as the model
    words = re.findall('(\w{4,10})', str(mi))
    hours = ['0','4','8','12','16','20'] * len(words)
    idx = [w + '_' + h for w,h in zip(words,hours)]
    idx = ['date']+idx
    RDF.columns = idx
    
    # filling holes
    for col in idx:
        RDF[col]= RDF[col].interpolate(method='linear', axis=0).ffill().bfill()
    
    # transforming wind direction
    direction = ['drct_0', 'drct_4', 'drct_8', 'drct_12', 'drct_16', 'drct_20']

    for col in direction:
        RDF[col]=RDF[col].apply(get_cos)
        
    #creatinf fiels for pilot rank
    RDF = pd.concat([RDF]*3, ignore_index=True)
    RDF['overall_rank_0']=0
    RDF['overall_rank_1']=0
    RDF['overall_rank_2']=0
    RDF['overall_rank_0'][0]=1
    RDF['overall_rank_0'][3]=1
    RDF['overall_rank_1'][1]=1
    RDF['overall_rank_1'][4]=1
    RDF['overall_rank_2'][2]=1
    RDF['overall_rank_2'][5]=1
    
    return RDF
    
    
        
def get_date(ts):
    return ts.date()

def get_hour(ts):
    return ts.hour
        
# create timespamp
def ts(epoch):
    return datetime.fromtimestamp(int(epoch)).strftime('%Y-%m-%d %H:%M:%S')
 

def get_cos(direction):
    return (cos(radians(direction)))**2


In [388]:
get_ready(hourly.content)

,date,alti_0,alti_4,alti_8,alti_12,alti_16,alti_20,drct_0,drct_4,drct_8,...,skyc1_20,tmpf_0,tmpf_4,tmpf_8,tmpf_12,tmpf_16,tmpf_20,overall_rank_0,overall_rank_1,overall_rank_2
0,2018-01-30,30.1925,30.21,30.0900,30.105,30.1300,30.1675,0.975528,0.994508,0.843121,...,71.75,40.75,40.25,44.00,46.0,44.25,41.75,1,0,0
1,2018-01-31,30.1925,30.21,30.2225,30.210,30.1975,30.2000,0.975528,0.994508,0.956773,...,99.00,40.75,40.25,41.75,44.0,43.50,43.00,0,1,0
2,2018-01-30,30.1925,30.21,30.0900,30.105,30.1300,30.1675,0.975528,0.994508,0.843121,...,71.75,40.75,40.25,44.00,46.0,44.25,41.75,0,0,1
3,2018-01-31,30.1925,30.21,30.2225,30.210,30.1975,30.2000,0.975528,0.994508,0.956773,...,99.00,40.75,40.25,41.75,44.0,43.50,43.00,1,0,0
4,2018-01-30,30.1925,30.21,30.0900,30.105,30.1300,30.1675,0.975528,0.994508,0.843121,...,71.75,40.75,40.25,44.00,46.0,44.25,41.75,0,1,0
5,2018-01-31,30.1925,30.21,30.2225,30.210,30.1975,30.2000,0.975528,0.994508,0.956773,...,99.00,40.75,40.25,41.75,44.0,43.50,43.00,0,0,1


### Testing new predictions
### Loading the model

In [339]:
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error,r2_score

In [332]:
conn = psycopg2.connect("dbname = soaring_predictor")
data = pd.read_sql('SELECT * from data;', conn)

In [363]:
features = ['overall_rank_0','overall_rank_1', 'overall_rank_2', 
            'alti_0', 'alti_4', 'alti_8', 'alti_12', 'alti_16',
            'drct_0', 'drct_4', 'drct_8', 'drct_12', 'drct_16',
            'dwpf_0', 'dwpf_4', 'dwpf_8', 'dwpf_12', 'dwpf_16',
            'p01i_0', 'p01i_4', 'p01i_8', 'p01i_12', 'p01i_16',
            'relh_0', 'relh_4', 'relh_8', 'relh_12', 'relh_16',
            'sknt_0', 'sknt_4', 'sknt_8', 'sknt_12', 'sknt_16',
            'skyc1_0', 'skyc1_4', 'skyc1_8', 'skyc1_12', 'skyc1_16',
            'psgr_0', 'psgr_4', 'psgr_8', 'psgr_12', 'psgr_16',
            'tmpf_0', 'tmpf_4', 'tmpf_8', 'tmpf_12', 'tmpf_16']

In [364]:
X = data[features]
y = data['left_perimeter']

In [371]:
paramGD = {'max_depth': 3,
 'max_features': 9,
 'min_samples_leaf': 5,
 'min_samples_split': 5,
 'n_estimators': 100,
 'n_jobs': -1}

paramMA = {'max_depth': 9,
 'max_features': 12,
 'min_samples_leaf': 3,
 'min_samples_split': 3,
 'n_estimators': 100,
 'n_jobs': -1}

paramXC = {'max_depth': 3,
 'max_features': 10,
 'min_samples_leaf': 3,
 'min_samples_split': 5,
 'n_estimators': 100,
 'n_jobs': -1}

In [366]:
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [373]:
y = data['max_alt']
X_train,X_test,y_train,y_test = train_test_split(X,y)
MA= RandomForestRegressor(**paramMA).fit(X_train,y_train)
predictions =MA.predict(X_test)
print('sq mean sq error: {}'.format( mean_squared_error(y_test,predictions)**0.5))
print('R sq : {}'.format( r2_score(y_test,predictions)))

sq mean sq error: 284.1131432004479
R sq : 0.42672659132732327


In [374]:
y = data['left_perimeter']
X_train,X_test,y_train,y_test = train_test_split(X,y)
GD= RandomForestClassifier(**paramGD).fit(X_train,y_train)
predictions = GD.predict(X_test)
# print('sq mean sq error: {}'.format( mean_squared_error(y_test,predictions)**0.5))
print('Accuracy : {}'.format(GD.score(X_test,y_test)))

Accuracy : 0.7520064205457464


In [375]:
y = data['XC']
X_train,X_test,y_train,y_test = train_test_split(X,y)
XC= RandomForestClassifier(**paramXC).fit(X_train,y_train)
predictions = XC.predict(X_test)
# print('sq mean sq error: {}'.format( mean_squared_error(y_test,predictions)**0.5))
print('Accuracy : {}'.format(XC.score(X_test,y_test)))

Accuracy : 0.7014446227929374


### Making the predictions

In [376]:
x= RDF[features]

In [392]:
GD.predict(x)

array([1, 0, 1, 1, 0, 0])

In [380]:
import pickle

with open('/Users/eduardodeangelis/Desktop/galvanize/soaring-predictor/test.pkl', 'rb') as f:
    model = pickle.load(f)

'/Users/eduardodeangelis/Desktop/galvanize/soaring-predictor/notebooks'